# Problem Description

Given two squares on a two dimensional plane, find a line that would cut these two squares in half. Assume that the top and bottom sides of the square run parallel to the x-axis.

# Solution

The code involves creating classes to represent points, lines, and squares, and then uses these classes to find the line that intersects both squares at their midpoints.

### Point Class

Represents a point on the graph with x and y coordinates.

In [2]:
class Point:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y

### Line Class

Represents a line defined by two points. It calculates the slope and y-intercept of the line.

In [3]:
class Line:
    def __init__(self, start: Point, end: Point) -> None:
        self.slope: float = 0
        self.yintercept: float = 0
        self.start: Point = start
        self.end: Point = end
        if start.x == end.x:
            self.slope = float('inf')  # PositiveInfinity
            self.yintercept = float('inf')  # PositiveInfinity
        else:
            self.slope = (end.y - start.y) / (end.x - start.x)
            self.yintercept = end.y - self.slope * end.x

### Square Class

Represents a square defined by its left, right, top, and bottom coordinates.
   * Middle(): Returns the middle point of the square.
   * Extend(): Extends a line segment connecting two middle points until it intersects the edge of the square.
   * Cut(): Finds the line that cuts both squares in half:
       * Calculates the middle points of both squares.
       * Uses Extend() to determine where the line segment intersects the edges of both squares.
       * Determines the start and end points of the line based on these intersections.

In [4]:
class Square:
    def __init__(self, left: float, right: float, top: float, bottom: float):
        self.left: float = left
        self.right: float = right
        self.top: float = top
        self.bottom: float = bottom
        self.size: float = right - left

    def Middle(self) -> Point:
        return Point((self.left + self.right) / 2.0,
                     (self.top + self.bottom) / 2.0)

    def Extend(self, mid1: Point, mid2: Point, size: float) -> Point:
        # Find what direction the line mid2 -> mid1 goes
        xdir: float = -1 if (mid1.x < mid2.x) else 1
        ydir: float = -1 if (mid1.y < mid2.y) else 1
        # If mid1 and mid2 have the same x value, then the slope calculation will
        # throw a divide by 0 exception. So, we compute this specially.
        if mid1.x == mid2.x:
            return Point(mid1.x, mid1.y + ydir * size / 2.0)
        slope: float = (mid1.y - mid2.y) / (mid1.x - mid2.x)
        x1: float = 0.0
        y1: float = 0.0
        # Calculate slope using the equation (y1-y2)/(x1-x2).
        # Note: If the slope is "steep" (>1) then the end of the line segment will
        # hit size/2 units away from the middle on the y axis. If the slope is
        # "shallow" (<1) the end of the line segment will hit size/2 units away
        # from the middle on the x axis.
        if abs(slope) == 1:
            x1 = mid1.x + xdir * size / 2.0
            y1 = mid1.y + ydir * size / 2.0
        elif abs(slope) < 1:  # shallow slope
            x1 = mid1.x + xdir * size / 2.0
            y1 = slope * (x1 - mid1.x) + mid1.y
        else:  # Steep slope
            y1 = mid1.y + ydir * size / 2.0
            x1 = (y1 - mid1.y) / slope + mid1.x
        return Point(x1, y1)

    def Cut(self, other: 'Square') -> Line:
        # Calculate where a line between each middle would collide with the edges of
        # the squares
        p1: Point = other.Extend(self.Middle(), other.Middle(), self.size)
        p2: Point = other.Extend(self.Middle(), other.Middle(), -self.size)
        p3: Point = other.Extend(other.Middle(), self.Middle(), other.size)
        p4: Point = other.Extend(other.Middle(), self.Middle(), -other.size)
        # Of above points, finds start and end of line. Start is farthest left (with
        # top most as a tie breaker) and end is farthest right (with bottom most as
        # a tie breaker)
        start: Point = p1
        end: Point = p1
        points: list[Point] = [p2, p3, p4]
        for point in points:
            if point.x < start.x or (point.x == start.x and point.y < start.y):
                start = point
            elif point.x > end.x or (point.x == end.x and point.y > end.y):
                end = point
        return Line(start, end)

## Example Usage

Here's how you can use these methods:

In [5]:
square1 = Square(1, 3, 3, 1)
square2 = Square(4, 6, 6, 4)

line = square1.Cut(square2)
print(f"Line starts at ({line.start.x}, {line.start.y}) and ends at ({line.end.x}, {line.end.y})")

Line starts at (1.0, 1.0) and ends at (6.0, 6.0)


# Literature

The contents base on the following literature:

* Gayle Laakmann McDowell, *Cracking the Coding Interview*, [Link](https://www.crackingthecodinginterview.com/).

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).